In [70]:
# Import our dependencies

import pandas as pd
from path import Path
from sqlalchemy import create_engine
#from config import db_password
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder



In [71]:
# Define directory path for files to keep from retyping every time
#file_dir = 'C://Users/kathy/Documents/Data_Analytics/Tennis_Analysis/'
file_dir = 'Resources/'
data = Path('Resources/atp_rankings_00s.csv')
df = pd.read_csv(data)
# Read in data file containing match data for specified year
tennis_stats_98_df = pd.read_csv(f'{file_dir}atp_matches_1998.csv', low_memory=False)
tennis_stats_99_df = pd.read_csv(f'{file_dir}atp_matches_1999.csv', low_memory=False)
tennis_stats_00_df = pd.read_csv(f'{file_dir}atp_matches_2000.csv', low_memory=False)

In [72]:
# Read in data file containing match data for specified year
tennis_stats_98_df = tennis_stats_98_df.assign(year = 1998)
tennis_stats_99_df = tennis_stats_99_df.assign(year = 1999)
tennis_stats_00_df = tennis_stats_00_df.assign(year = 2000)

In [73]:
# Concatenate 98, 99, and 00 files into one dataframe
tennis_stats_df = pd.concat([tennis_stats_98_df, tennis_stats_99_df, tennis_stats_00_df])

In [74]:
# Display list of column names sorted
sorted(tennis_stats_df.columns.tolist())

['best_of',
 'draw_size',
 'l_1stIn',
 'l_1stWon',
 'l_2ndWon',
 'l_SvGms',
 'l_ace',
 'l_bpFaced',
 'l_bpSaved',
 'l_df',
 'l_svpt',
 'loser_age',
 'loser_entry',
 'loser_hand',
 'loser_ht',
 'loser_id',
 'loser_ioc',
 'loser_name',
 'loser_rank',
 'loser_rank_points',
 'loser_seed',
 'match_num',
 'minutes',
 'round',
 'score',
 'surface',
 'tourney_date',
 'tourney_id',
 'tourney_level',
 'tourney_name',
 'w_1stIn',
 'w_1stWon',
 'w_2ndWon',
 'w_SvGms',
 'w_ace',
 'w_bpFaced',
 'w_bpSaved',
 'w_df',
 'w_svpt',
 'winner_age',
 'winner_entry',
 'winner_hand',
 'winner_ht',
 'winner_id',
 'winner_ioc',
 'winner_name',
 'winner_rank',
 'winner_rank_points',
 'winner_seed',
 'year']

In [75]:
# Remove unnecessary columns
tennis_stats_df.drop(columns=['best_of',
                             'draw_size',
                             'loser_entry',
                             'loser_hand',
                             'loser_ht',
                             'loser_ioc',
                             'loser_name',
                             'loser_rank',
                             'loser_rank_points',
                             'loser_seed',
                             'match_num',
                             'minutes',
                             'round',
                             'score',
                             'surface',
                             'tourney_date',
                             'tourney_id',
                             'tourney_level',
                             'tourney_name',
                             'winner_entry',
                              'winner_hand',
                              'winner_ht',
                              'winner_ioc',
                              'winner_name',
                              'winner_rank',
                              'winner_rank_points',
                              'winner_seed'
], inplace=True)
tennis_stats_df.head()

,winner_id,winner_age,loser_id,loser_age,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,...,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,year
0,102035,25.7,101917,26.6,7.0,1.0,56.0,29.0,22.0,17.0,...,2.0,4.0,54.0,29.0,18.0,11.0,9.0,2.0,5.0,1998
1,101727,27.7,102548,22.8,6.0,0.0,56.0,30.0,23.0,15.0,...,1.0,7.0,64.0,32.0,19.0,11.0,8.0,9.0,13.0,1998
2,102765,21.7,102491,23.0,12.0,0.0,51.0,28.0,22.0,9.0,...,3.0,2.0,60.0,38.0,18.0,7.0,9.0,5.0,12.0,1998
3,102563,22.7,101647,28.2,6.0,1.0,63.0,30.0,20.0,19.0,...,3.0,2.0,56.0,37.0,21.0,10.0,11.0,1.0,6.0,1998
4,102796,21.6,102104,25.3,9.0,6.0,80.0,27.0,22.0,23.0,...,5.0,5.0,99.0,57.0,33.0,20.0,13.0,12.0,17.0,1998


In [76]:
# Make a dopy of the tennis_stats dataframe
winners_df = tennis_stats_df.copy()

In [77]:
# Create winners dataframe 
winners_df.drop(columns=['loser_age',
                         'l_1stIn',
                         'l_1stWon', 
                         'l_2ndWon',
                         'l_SvGms',
                         'l_ace',
                         'l_bpFaced',
                         'l_bpSaved',
                         'l_df',
                         'l_svpt',
                         'loser_id'], inplace=True)
winners_df.head()

,winner_id,winner_age,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,year
0,102035,25.7,7.0,1.0,56.0,29.0,22.0,17.0,9.0,1.0,1.0,1998
1,101727,27.7,6.0,0.0,56.0,30.0,23.0,15.0,9.0,1.0,2.0,1998
2,102765,21.7,12.0,0.0,51.0,28.0,22.0,9.0,9.0,4.0,7.0,1998
3,102563,22.7,6.0,1.0,63.0,30.0,20.0,19.0,10.0,3.0,5.0,1998
4,102796,21.6,9.0,6.0,80.0,27.0,22.0,23.0,13.0,3.0,8.0,1998


In [78]:
# Add winner column to winners_df with a value of 1
winners_df['result'] = 1
winners_df.head()

,winner_id,winner_age,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,year,result
0,102035,25.7,7.0,1.0,56.0,29.0,22.0,17.0,9.0,1.0,1.0,1998,1
1,101727,27.7,6.0,0.0,56.0,30.0,23.0,15.0,9.0,1.0,2.0,1998,1
2,102765,21.7,12.0,0.0,51.0,28.0,22.0,9.0,9.0,4.0,7.0,1998,1
3,102563,22.7,6.0,1.0,63.0,30.0,20.0,19.0,10.0,3.0,5.0,1998,1
4,102796,21.6,9.0,6.0,80.0,27.0,22.0,23.0,13.0,3.0,8.0,1998,1


In [79]:
# Rename columns in winners_df
winners_df.rename(columns={"winner_age":"age",
                           "w_ace":"ace",
                           "w_df":"df",
                           "w_svpt":"svpt",
                           "w_1stIn":"1stIn",
                           "w_1stWon":"1stWon",
                           "w_2ndWon":"2ndWon",
                           "w_SvGms":"svGms",
                           "w_bpFaced":"bpFaced",
                           "w_bpSaved":"bpSaved",
                           "winner_id":"player_id"}, inplace=True)
winners_df.head()

,player_id,age,ace,df,svpt,1stIn,1stWon,2ndWon,svGms,bpSaved,bpFaced,year,result
0,102035,25.7,7.0,1.0,56.0,29.0,22.0,17.0,9.0,1.0,1.0,1998,1
1,101727,27.7,6.0,0.0,56.0,30.0,23.0,15.0,9.0,1.0,2.0,1998,1
2,102765,21.7,12.0,0.0,51.0,28.0,22.0,9.0,9.0,4.0,7.0,1998,1
3,102563,22.7,6.0,1.0,63.0,30.0,20.0,19.0,10.0,3.0,5.0,1998,1
4,102796,21.6,9.0,6.0,80.0,27.0,22.0,23.0,13.0,3.0,8.0,1998,1


In [80]:
# Make a dopy of the tennis_stats dataframe
losers_df = tennis_stats_df.copy()

In [81]:
# Create losers dataframe
losers_df.drop(columns=['winner_age',
                         'w_1stIn',
                         'w_1stWon', 
                         'w_2ndWon',
                         'w_SvGms',
                         'w_ace',
                         'w_bpFaced',
                         'w_bpSaved',
                         'w_df',
                         'w_svpt',
                         'winner_id'], inplace=True)
losers_df.head()

,loser_id,loser_age,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,year
0,101917,26.6,2.0,4.0,54.0,29.0,18.0,11.0,9.0,2.0,5.0,1998
1,102548,22.8,1.0,7.0,64.0,32.0,19.0,11.0,8.0,9.0,13.0,1998
2,102491,23.0,3.0,2.0,60.0,38.0,18.0,7.0,9.0,5.0,12.0,1998
3,101647,28.2,3.0,2.0,56.0,37.0,21.0,10.0,11.0,1.0,6.0,1998
4,102104,25.3,5.0,5.0,99.0,57.0,33.0,20.0,13.0,12.0,17.0,1998


In [82]:
# Add loser column to winners_df with a value of 0
losers_df['result'] = 0
losers_df.head()

,loser_id,loser_age,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,year,result
0,101917,26.6,2.0,4.0,54.0,29.0,18.0,11.0,9.0,2.0,5.0,1998,0
1,102548,22.8,1.0,7.0,64.0,32.0,19.0,11.0,8.0,9.0,13.0,1998,0
2,102491,23.0,3.0,2.0,60.0,38.0,18.0,7.0,9.0,5.0,12.0,1998,0
3,101647,28.2,3.0,2.0,56.0,37.0,21.0,10.0,11.0,1.0,6.0,1998,0
4,102104,25.3,5.0,5.0,99.0,57.0,33.0,20.0,13.0,12.0,17.0,1998,0


In [83]:
# Rename columns in losers_df
losers_df.rename(columns={"loser_age":"age",
                           "l_ace":"ace",
                           "l_df":"df",
                           "l_svpt":"svpt",
                           "l_1stIn":"1stIn",
                           "l_1stWon":"1stWon",
                           "l_2ndWon":"2ndWon",
                           "l_SvGms":"svGms",
                           "l_bpFaced":"bpFaced",
                           "l_bpSaved":"bpSaved",
                           "loser_id":"player_id"}, inplace=True)
losers_df.head()

,player_id,age,ace,df,svpt,1stIn,1stWon,2ndWon,svGms,bpSaved,bpFaced,year,result
0,101917,26.6,2.0,4.0,54.0,29.0,18.0,11.0,9.0,2.0,5.0,1998,0
1,102548,22.8,1.0,7.0,64.0,32.0,19.0,11.0,8.0,9.0,13.0,1998,0
2,102491,23.0,3.0,2.0,60.0,38.0,18.0,7.0,9.0,5.0,12.0,1998,0
3,101647,28.2,3.0,2.0,56.0,37.0,21.0,10.0,11.0,1.0,6.0,1998,0
4,102104,25.3,5.0,5.0,99.0,57.0,33.0,20.0,13.0,12.0,17.0,1998,0


In [85]:
# Concatenate two (winners and losers) dataframes together
match_stats_df = pd.concat([winners_df, losers_df])
cleaned_match_stats = match_stats_df.dropna()
cleaned_match_stats.head()

,player_id,age,ace,df,svpt,1stIn,1stWon,2ndWon,svGms,bpSaved,bpFaced,year,result
0,102035,25.7,7.0,1.0,56.0,29.0,22.0,17.0,9.0,1.0,1.0,1998,1
1,101727,27.7,6.0,0.0,56.0,30.0,23.0,15.0,9.0,1.0,2.0,1998,1
2,102765,21.7,12.0,0.0,51.0,28.0,22.0,9.0,9.0,4.0,7.0,1998,1
3,102563,22.7,6.0,1.0,63.0,30.0,20.0,19.0,10.0,3.0,5.0,1998,1
4,102796,21.6,9.0,6.0,80.0,27.0,22.0,23.0,13.0,3.0,8.0,1998,1


In [86]:
len(cleaned_match_stats)

18220

In [87]:
cleaned_match_stats

,player_id,age,ace,df,svpt,1stIn,1stWon,2ndWon,svGms,bpSaved,bpFaced,year,result
0,102035,25.7,7.0,1.0,56.0,29.0,22.0,17.0,9.0,1.0,1.0,1998,1
1,101727,27.7,6.0,0.0,56.0,30.0,23.0,15.0,9.0,1.0,2.0,1998,1
2,102765,21.7,12.0,0.0,51.0,28.0,22.0,9.0,9.0,4.0,7.0,1998,1
3,102563,22.7,6.0,1.0,63.0,30.0,20.0,19.0,10.0,3.0,5.0,1998,1
4,102796,21.6,9.0,6.0,80.0,27.0,22.0,23.0,13.0,3.0,8.0,1998,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2974,102538,24.8,4.0,6.0,96.0,46.0,31.0,22.0,16.0,3.0,9.0,2000,0
2975,102106,27.3,14.0,0.0,96.0,59.0,39.0,23.0,15.0,3.0,6.0,2000,0
2976,210013,27.4,12.0,0.0,92.0,50.0,37.0,21.0,14.0,5.0,8.0,2000,0
2977,101611,30.5,10.0,4.0,105.0,48.0,37.0,33.0,16.0,3.0,5.0,2000,0


In [88]:
y = cleaned_match_stats["result"]
X = cleaned_match_stats.drop(columns="result")

In [89]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(13665, 12)

In [90]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)

In [91]:
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [92]:
cleaned_match_stats.columns.values

array(['player_id', 'age', 'ace', 'df', 'svpt', '1stIn', '1stWon',
       '2ndWon', 'svGms', 'bpSaved', 'bpFaced', 'year', 'result'],
      dtype=object)

In [93]:
y_pred = classifier.predict(X_test)
results_cleaned_match_stats = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results_cleaned_match_stats.head(20)

,Prediction,Actual
0,0,0
1,1,1
2,1,1
3,1,1
4,0,0
5,1,1
6,0,0
7,0,0
8,0,0
9,1,1


In [94]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.8019758507135016


In [95]:

# Decision Tree Model

In [96]:
# Define features set
X = cleaned_match_stats.copy()
X = X.drop("result", axis=1)
X.head()

,player_id,age,ace,df,svpt,1stIn,1stWon,2ndWon,svGms,bpSaved,bpFaced,year
0,102035,25.7,7.0,1.0,56.0,29.0,22.0,17.0,9.0,1.0,1.0,1998
1,101727,27.7,6.0,0.0,56.0,30.0,23.0,15.0,9.0,1.0,2.0,1998
2,102765,21.7,12.0,0.0,51.0,28.0,22.0,9.0,9.0,4.0,7.0,1998
3,102563,22.7,6.0,1.0,63.0,30.0,20.0,19.0,10.0,3.0,5.0,1998
4,102796,21.6,9.0,6.0,80.0,27.0,22.0,23.0,13.0,3.0,8.0,1998


In [97]:
# Define target vector
y = cleaned_match_stats["result"].values.reshape(-1, 1)
y[:5]

array([[1],
       [1],
       [1],
       [1],
       [1]])

In [98]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [99]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(13665, 12)
(4555, 12)
(13665, 1)
(4555, 1)


In [100]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state=78, train_size=0.80)

In [101]:
print(X_train2.shape)
print(X_test2.shape)
print(y_train2.shape)
print(y_test2.shape)

(14576, 12)
(3644, 12)
(14576, 1)
(3644, 1)


In [102]:
# Creating StandardScaler instance
scaler = StandardScaler()

In [103]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

In [104]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [105]:
## Fitting the Decision Tree Model
# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

In [106]:
# Fitting the model
model = model.fit(X_train_scaled, y_train)

In [107]:
# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

In [108]:
# Model Evaluation
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [109]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))


Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1617,650
Actual 1,643,1645


Accuracy Score : 0.7161361141602635
Classification Report
              precision    recall  f1-score   support

           0       0.72      0.71      0.71      2267
           1       0.72      0.72      0.72      2288

    accuracy                           0.72      4555
   macro avg       0.72      0.72      0.72      4555
weighted avg       0.72      0.72      0.72      4555

